In [ ]:
import os, sys
project_root_dir = os.path.dirname('.')
sparse_dir = os.path.join(project_root_dir, 'modules/Sparse')
if sparse_dir not in sys.path:
    sys.path.append(sparse_dir)

import torch
from torch import nn
import numpy as np

In [ ]:
# from torch.nn.parameter import Parameter
# import torch.nn.functional as F
# from Sparse.modules.variational import VariationalLayer

# class LinearSVD(nn.Linear, VariationalLayer):
#     def __init__(self, in_features, out_features, p_threshold = 0.952572, bias=True) -> None:
#         r'''
#             Parameters
#             ----------
#                 in_features: int,
#                     Number of input features.

#                 out_features: int,
#                     Number of output features.
                
#                 p_threshold: float,
#                     It consists in the \rho (binary dropout rate) threshold used in order to discard the weight.
#                     In this approach, an Gaussian Dropout is being used which std is \alpha = \rho/(1-\rho) so, 
#                     Infinitely large \sigma_{ij} corresponds to infinitely large multiplicative noise in w_{ij}. By 
#                     default, the threshold is set to 0.952572 (\log(\sigma) ~ 3).

#                 bias: bool,
#                     If True, adds a bias term to the output.
#         '''
#         super(LinearSVD, self).__init__(in_features, out_features, bias)
    
#         self.log_alpha_threshold = np.log(p_threshold / (1-p_threshold))
#         self.log_sigma = Parameter(1e-2 * torch.randn(out_features, in_features))

#         self.log_sigma.data.fill_(-5) # Initialization based on the paper, Figure 1
    
#     def forward(self, x: torch.Tensor) -> torch.Tensor:
#         self.log_alpha = self.compute_log_alpha(self.log_sigma, torch.abs(self.weight))

#         if self.training:
#             # LRT = local reparametrization trick (For details, see https://arxiv.org/pdf/1506.02557.pdf)
#             lrt_mean =  F.linear(x, self.weight, self.bias)
#             lrt_std = torch.sqrt(F.linear(x * x, torch.exp(self.log_sigma * 2.0)) + 1e-8)#.unsqueeze(dim=1)
#             eps = torch.normal(0, torch.ones_like(lrt_std))
#             return lrt_mean + lrt_std * eps
        
#         return F.linear(x, self.weight * (self.log_alpha < self.log_alpha_threshold).float(), self.bias)

#     def kl_reg(self):
#         k1, k2, k3 = torch.Tensor([0.63576]).cuda(), torch.Tensor([1.8732]).cuda(), torch.Tensor([1.48695]).cuda()
#         kl = k1 * torch.sigmoid(k2 + k3 * self.log_alpha) - 0.5 * torch.log1p(torch.exp(-self.log_alpha))
#         return -(torch.sum(kl))

#     def compute_log_alpha(self, log_sigma, theta):
#         r''' 
#             Compute the log \alpha values from \theta and log \sigma^2.

#             The relationship between \sigma^2, \theta, and \alpha as defined in the
#             paper https://arxiv.org/abs/1701.05369 is \sigma^2 = \alpha * \theta^2.

#             This method calculates the log \alpha values based on this relation:
#                 \log(\alpha) = 2*\log(\sigma) - 2*\log(\theta)
#         ''' 
#         log_alpha = log_sigma * 2.0 - 2.0 * torch.log(1e-16 + torch.abs(theta))
#         log_alpha = torch.clamp(log_alpha, -10, 10) # clipping for a numerical stability
#         return log_alpha

In [ ]:
def compute_log_alpha(log_sigma, theta):
        r''' 
            Compute the log \alpha values from \theta and log \sigma^2.

            The relationship between \sigma^2, \theta, and \alpha as defined in the
            paper https://arxiv.org/abs/1701.05369 is \sigma^2 = \alpha * \theta^2.

            This method calculates the log \alpha values based on this relation:
                \log(\alpha) = 2*\log(\sigma) - 2*\log(\theta)
        ''' 
        log_alpha = log_sigma * 2.0 - 2.0 * torch.log(1e-16 + torch.abs(theta))
        log_alpha = torch.clamp(log_alpha, -10, 10) # clipping for a numerical stability
        return log_alpha

from Sparse.modules.variational import VariationalLayer
from torch.nn.parameter import Parameter

class FeatureSelectionVariationalDropout(nn.Module, VariationalLayer):
    def __init__(self, in_features: int, p_threshold:float = 0.952572) -> None:
        super(FeatureSelectionVariationalDropout, self).__init__()
        self.in_features = in_features
        self.log_alpha_threshold = np.log(p_threshold / (1-p_threshold))
        self.log_sigma = Parameter(1e-2 * torch.randn(in_features))
        self.weight = Parameter(torch.randn(in_features))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        self.log_alpha = compute_log_alpha(self.log_sigma, torch.abs(self.weight))
        
        if self.training:
            # LRT = local reparametrization trick (For details, see https://arxiv.org/pdf/1506.02557.pdf)
            lrt_mean =  x * self.weight
            lrt_std = torch.sqrt((x*x) * torch.exp(self.log_sigma * 2.0) + 1e-8)
            eps = torch.normal(0, torch.ones_like(lrt_std))
            return lrt_mean + lrt_std * eps
        
        return x * self.weight * (self.log_alpha < self.log_alpha_threshold).float()

    def kl_reg(self):
        k1, k2, k3 = torch.Tensor([0.63576]).cuda(), torch.Tensor([1.8732]).cuda(), torch.Tensor([1.48695]).cuda()
        kl = k1 * torch.sigmoid(k2 + k3 * self.log_alpha) - 0.5 * torch.log1p(torch.exp(-self.log_alpha))
        return -(torch.sum(kl))

In [ ]:
from Sparse.modules.variational.utils import SGVBL
from torch import nn
import torch

class Model(nn.Module):
    def __init__(self,in_features: int, nb_features: int, threshold: float = .95):
        super(Model, self).__init__()

        if threshold < 0. or threshold > 1.:
            raise ValueError('threshold must be between 0 and 1')

        self.model = nn.Sequential(
            FeatureSelectionVariationalDropout(in_features, p_threshold=threshold),
            nn.Linear(in_features, nb_features),
            nn.ReLU(),
            nn.Linear(nb_features, nb_features//2),
            nn.Dropout(p=0.2),
            nn.ReLU(),
            nn.Linear(nb_features//2, nb_features//4),
            nn.Dropout(p=0.2),
            nn.ReLU(),
            nn.Linear(nb_features//4, 2)
        )
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.model(x)

In [ ]:
from dataset import FeaturesDataset
import numpy as np

dataset = FeaturesDataset('data/dataset/', normalize=True)

In [ ]:
from tqdm import tqdm
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

def train(model, dataset, batch_size = 128, n_epochs=10, log_dir='log/fs_vd'):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = SGVBL(model, len(dataset)).to(device)

    logger = SummaryWriter(log_dir)

    # weighted sampler
    samples = dataset.dataset.y[dataset.indices]
    class_weight = [1/(samples == 0).sum(), 1/(samples == 1).sum()]
    samples_weight = np.zeros(len(dataset))
    samples_weight[samples == 0] = class_weight[0]
    samples_weight[samples == 1] = class_weight[1]
    
    sampler = torch.utils.data.WeightedRandomSampler(samples_weight, len(dataset))
    loader = DataLoader(dataset, batch_size=batch_size, sampler=sampler)
    reg = 1e-6

    epoch_iterator = tqdm(
            range(n_epochs),
            leave=True,
            unit="epoch",
            postfix={"tls": "%.4f" % 1},
        )

    for epoch in epoch_iterator:
        reg = min(reg + 2.5e-3, 1)
        logger.add_scalar('kl_reg', reg, epoch)

        train_loss, train_acc = 0, 0 
        for idx, (inputs, targets) in enumerate(loader):
            optimizer.zero_grad()

            inputs = inputs.to(device)
            targets = targets.to(device)
            outputs = model(inputs)

            loss = criterion(outputs, targets, reg)
            loss.backward()
            optimizer.step()

            # Log
            pred = outputs.data.max(1)[1]
            train_loss += float(loss)

            train_acc += np.sum(np.equal(pred.cpu().numpy(), targets.cpu().data.numpy()))

            if idx % 10 == 0:
                epoch_iterator.set_postfix(tls="%.4f" % loss.item())
        
        logger.add_scalar('Loss', train_loss / len(dataset), epoch)
        logger.add_scalar('Accuracy', train_acc / len(dataset) * 100, epoch)

        for i, c in enumerate(model.model.children()):
            if hasattr(c, 'kl_reg'):
                logger.add_scalar('sp_%s' % i, (c.log_alpha.cpu().data.numpy() > c.log_alpha_threshold).sum(), epoch)
    
    return model

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

n_features = len(dataset.features)

k_folds = 5
kfold = KFold(n_splits=k_folds, shuffle=True)

features_importance = []
model_accuracy = []

for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
    train_set = torch.utils.data.Subset(dataset, train_ids)
    test_set = torch.utils.data.Subset(dataset, test_ids)

    model = Model(n_features, 256, threshold=.9)
    model = train(model, train_set, batch_size=32, n_epochs=500, log_dir='log/fs_vd/{}'.format(fold))

    model.eval()
    y_pred = []
    y_true = []

    test_loader = DataLoader(test_set, batch_size=64, shuffle=False)
    for inputs, targets in test_loader:
        pred = model(inputs.cuda())
        pred = pred.argmax(dim=1)
        y_pred.append(pred.cpu())
        y_true.append(targets)

    y_pred = torch.cat(y_pred)
    y_true = torch.cat(y_true)
    model_accuracy.append(accuracy_score(y_true, y_pred))
    features_importance.append(torch.sigmoid(model.model[0].log_alpha).cpu().detach().numpy())
    print(model_accuracy[-1])

In [ ]:
# Mean accuracy
print('Mean accuracy: {}'.format(np.mean(model_accuracy)))

In [ ]:
import pandas as pd

features_importance_ = torch.tensor(np.array(features_importance)).mean(axis=0)
features_score, index = features_importance_.sort()
features_names = np.array(dataset.features)[index.cpu()]

features_importance_df = pd.DataFrame(features_importance_[index], index=features_names, columns=['Importance'])
features_importance_df.index.name = 'Features'

features_importance_df.to_csv('data/features_importance/variational_dropout.csv')

features_importance_df